In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord

In [2]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


In [3]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    # convert notes to one-hot encoded
    one_hot_notes = []
    for note in notes:
        one_hot_note = np.zeros(n_vocab)
        one_hot_note[note_to_int[note]] = 1
        one_hot_notes.append(one_hot_note)

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(one_hot_notes) - sequence_length, 1):
        sequence_in = one_hot_notes[i:i + sequence_length]
        sequence_out = one_hot_notes[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, n_vocab))

    network_output = np.array(network_output)

    return (network_input, network_output)

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
/Users/masatoshi/pyenvs/music/lib/python3.6/site-packages/matplotlib/style/core.py:201: UserWarning: In /Users/masatoshi/pyenvs/music/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  warnings.warn(message)


In [5]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [6]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)


In [9]:
from keras.utils import plot_model

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    print(network_input.shape)
    print(network_output.shape)
    model = create_network(network_input, n_vocab)
    plot_model(model, to_file='lstm_midi.png', show_shapes=True)

    train(model, network_input, network_output)

In [10]:
train_network()

Parsing midi_songs/bwv782.mid
Parsing midi_songs/bwv783.mid
Parsing midi_songs/bwv781.mid
Parsing midi_songs/bwv780.mid
Parsing midi_songs/bwv784.mid
Parsing midi_songs/bwv785.mid
Parsing midi_songs/bwv778.mid
Parsing midi_songs/bwv786.mid
Parsing midi_songs/bwv779.mid
Parsing midi_songs/bwv774.mid
Parsing midi_songs/bwv775.mid
Parsing midi_songs/bwv777.mid
Parsing midi_songs/bwv776.mid
Parsing midi_songs/bwv772.mid
Parsing midi_songs/bwv773.mid
(8327, 100, 124)
(8327, 124)
Epoch 1/200
1536/8327 [====>.........................] - ETA: 14:27 - loss: 4.5239

KeyboardInterrupt: 

In [16]:
!pip install pydot

    100% |████████████████████████████████| 133kB 849kB/s ta 0:00:01
  Running setup.py bdist_wheel for pydot ... done
  Stored in directory: /Users/masatoshi/Library/Caches/pip/wheels/6a/a5/14/25541ebcdeaf97a37b6d05c7ff15f5bd20f5e91b99d313e5b4
Successfully built pydot
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
import pydot